<a href="https://colab.research.google.com/github/CatalinaMaya/Caso_Finanzas/blob/main/Caso_Seguros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data ingestion**

In [ ]:
#Librerías
import pandas as pd
import numpy as np
import string 
import plotly.express as px # gráficos
import plotly.graph_objs as go # gráficos
import seaborn as sns # gráficos
from sklearn.impute import SimpleImputer # para imputación
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn import linear_model ## para regresión lineal
from sklearn import tree ###para ajustar arboles de decisión
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor ##Ensamble con bagging
from sklearn.ensemble import GradientBoostingRegressor ###Ensamble boosting
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt ### gráficos
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer ### para imputación
from sklearn import linear_model ## para regresión lineal
from sklearn import tree ###para ajustar arboles de decisión
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor ##Ensamble con bagging
from sklearn.ensemble import GradientBoostingRegressor ###Ensamble boosting
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt ### gráficos

In [ ]:
#Conexión con colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Cargar datos
siniestros = pd.read_csv('/content/drive/Shareddrives/Analítica/NIVEL 3/Finanzas/DB_Siniestros.txt', encoding="ISO-8859-1", sep=None, engine='python')
sociodemograficas = pd.read_csv('/content/drive/Shareddrives/Analítica/NIVEL 3/Finanzas/BD_Sociodemograficas.txt', encoding="ISO-8859-1", sep=None, engine='python')
#expuestos1 = pd.read_csv('/content/drive/Shareddrives/Analítica/NIVEL 3/Finanzas/BD_Expuestos.txt', encoding="ISO-8859-1", sep=None, engine='python')

In [ ]:
expuestos = pd.read_csv("/content/drive/Shareddrives/Analítica/NIVEL 3/Finanzas/BD_expuestos1.xlsx - Hoja1.csv")

In [ ]:
expuestos1 = pd.read_csv('/content/drive/Shareddrives/Analítica/NIVEL 3/Finanzas/BD_Expuestos.txt', encoding="ISO-8859-1", sep=None, engine='python')

In [ ]:
expuestos1

,Asegurado_Id,Poliza_Asegurado_Id,FECHA_INICIO,FECHA_CANCELACION,FECHA_FIN
0,16286173,125696117,2018/01/01,2018/04/10,2018/04/10
1,11885152,141320760,2018/01/01,NaN,2019/12/31
2,763957,162532851,2018/01/01,NaN,2019/12/31
3,29082204,154620473,2018/01/01,2018/04/19,2018/04/19
4,4567640,93633862,2018/01/01,NaN,2019/12/31
...,...,...,...,...,...
300895,28476983,53646742,2018/01/01,NaN,2019/12/31
300896,8479396,89963102,2018/01/01,2018/03/08,2018/03/08
300897,6598270,165960557,2018/02/28,NaN,2019/12/31
300898,1000907,105811540,2018/01/01,NaN,2019/12/31


# **Limpieza de datos**

## Base de datos siniestros

In [ ]:
def percent_null(datos):
    shape = datos.shape
    lista = datos.columns
    for i in lista:
        nulos = datos[i].isnull().sum()
        print(i, "->", nulos, " | ",round((nulos/shape[0])*100,2), "%")

In [ ]:
percent_null(siniestros)

Mes_Pago -> 0  |  0.0 %
Asegurado_Id -> 0  |  0.0 %
Reclamacion -> 0  |  0.0 %
Diagnostico_Codigo -> 0  |  0.0 %
Diagnostico_Desc -> 0  |  0.0 %
Eventos -> 0  |  0.0 %
Valor_Pagado -> 0  |  0.0 %


In [ ]:
lista = siniestros.columns
for i in lista:
    print(i)
    print(siniestros[i].value_counts())
    print('__________________________')

Mes_Pago
201907    155621
201910    155423
201912    155292
201909    153168
201905    146671
201903    146252
201810    145425
201911    144268
201908    143317
201812    142861
201904    140957
201902    139042
201906    137343
201808    136323
201804    135750
201805    135603
201811    133916
201809    133032
201806    131197
201802    129596
201807    127479
201803    126558
201901    113847
201801     99539
Name: Mes_Pago, dtype: int64
__________________________
Asegurado_Id
5147267     143
3458963     127
1224455     122
12814942    119
9508235     118
           ... 
24702041      1
29243926      1
41152319      1
3321810       1
30824331      1
Name: Asegurado_Id, Length: 237167, dtype: int64
__________________________
Reclamacion
CONSULTA EXTERNA                                                   1255793
EXAMENES DE DIAGNOSTICO                                             694238
LABORATORIO CLINICO                                                 520214
CONSULTA DE URGENCIAS    

In [ ]:
siniestros.dtypes

Mes_Pago                int64
Asegurado_Id            int64
Reclamacion            object
Diagnostico_Codigo     object
Diagnostico_Desc       object
Eventos                 int64
Valor_Pagado          float64
dtype: object

Clasificación internacional de enfermedades:
https://es.wikipedia.org/wiki/CIE-10
https://www.sssalud.gob.ar/hospitales/archivos/cie_10_revi.pdf

In [ ]:
siniestros[siniestros.Diagnostico_Codigo.str.match(pat = 'A')]['Diagnostico_Desc'].unique()

array(['VERRUGAS (VENÉREAS) ANOGENITALES',
       'FIEBRE DEL DENGUE HEMORRÁGICO', 'FIEBRES RECURRENTES',
       'DIARREA Y GASTROENTERITIS DE PRESUNTO ORIGEN INFECCIOSO',
       'SEPTICEMIA, NO ESPECIFICADA',
       'FIEBRE RECURRENTE, NO ESPECIFICADA',
       'LESIONES PAPILOMATOSAS MÚLTIPLES Y FRAMBESIA CON PASO DE CANGREJO',
       'TUBERCULOSIS DEL PULMÓN, CON EXAMEN BACTERIOLÓGICO E HISTOLÓGICO NEGATIVOS',
       'FIEBRE DEL DENGUE [DENGUE CLÁSICO]',
       'FIEBRE EXANTEMÁTICA ENTEROVIRAL [EXANTEMA DE BOSTON]',
       'INFECCIÓN DEBIDA A ESCHERICHIA COLI ENTEROINVASIVA',
       'INFECCIÓN GONOCÓCICA DEL ANO Y DEL RECTO',
       'AMEBIASIS INTESTINAL CRÓNICA',
       'INFECCIÓN INTESTINAL BACTERIANA, NO ESPECIFICADA',
       'MENINGITIS VIRAL', 'VÉRTIGO EPIDÉMICO', 'OTRAS SEPTICEMIAS',
       'COLITIS AMEBIANA NO DISENTÉRICA', 'ENTERITIS DEBIDA A ADENOVIRUS',
       'ENTERITIS DEBIDA A SALMONELLA',
       'OTRAS TUBERCULOSIS RESPIRATORIAS, CONFIRMADAS BACTERIOLÓGICA E HISTOLÓGICA

In [ ]:
siniestros[siniestros.Diagnostico_Codigo.str.isnumeric()]['Diagnostico_Desc'].unique()

array(['DIAGNÓSTICO PENDIENTE'], dtype=object)

In [ ]:
string.ascii_uppercase 

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [ ]:
lista = []
lista_abc =list(string.ascii_uppercase )

for letter in lista_abc:
    cantidad = siniestros.Diagnostico_Codigo.str.match(pat =letter).sum()
    lista.append(cantidad)

In [ ]:
lista_abc.append('numeric')
lista.append(siniestros.Diagnostico_Codigo.str.isnumeric().sum())

In [ ]:
diccionario = {
    'letra': lista_abc,
    'cantidad': lista
}

In [ ]:
cie = pd.DataFrame(diccionario).sort_values(by= 'cantidad', ascending = False)

In [ ]:
cie[cie['letra'] == 'numeric']

,letra,cantidad
26,numeric,2740851


In [ ]:
numeric = siniestros[siniestros['Diagnostico_Codigo'] == '9']

In [ ]:
numeric['Reclamacion'].value_counts()

CONSULTA EXTERNA                                                   1255793
LABORATORIO CLINICO                                                 520214
EXAMENES DE DIAGNOSTICO                                             423432
CONSULTA DE URGENCIAS                                               365689
FISIOTERAPIA ILIMITADA                                               98370
CONSULTAS PRENATALES                                                 33685
TRATAMIENTO MEDICO AMBULATORIO POR ACCIDENTES                        16314
INFILTRACIONES                                                       11584
Sin Informacion                                                       8802
CONCILIACION                                                          4391
TRATAMIENTO AMBULATORIO POR CANCER                                     633
CIRUGIA AMBULATORIA POR ENFERMEDAD                                     612
CIRUGIA AMBULATORIA POR ACCIDENTE                                      607
PAMD                     

In [ ]:
numeric['Diagnostico_Desc'].value_counts()

DIAGNÓSTICO PENDIENTE    2740383
Name: Diagnostico_Desc, dtype: int64

In [ ]:
siniestros[siniestros['Asegurado_Id'] == 5147267]

,Mes_Pago,Asegurado_Id,Reclamacion,Diagnostico_Codigo,Diagnostico_Desc,Eventos,Valor_Pagado
32100,201809,5147267,CONSULTA EXTERNA,9,DIAGNÓSTICO PENDIENTE,1,60760.00
75388,201809,5147267,LABORATORIO CLINICO,9,DIAGNÓSTICO PENDIENTE,8,571824.76
84077,201802,5147267,CIRUGIA AMBULATORIA POR ENFERMEDAD,C349,"TUMOR MALIGNO DE LOS BRONQUIOS O DEL PULMÓN, P...",1,5271850.08
103941,201812,5147267,EXAMENES DE DIAGNOSTICO,9,DIAGNÓSTICO PENDIENTE,3,1959144.20
240331,201801,5147267,LABORATORIO CLINICO,9,DIAGNÓSTICO PENDIENTE,9,90394.76
...,...,...,...,...,...,...,...
3252509,201904,5147267,PAMD,C349,"TUMOR MALIGNO DE LOS BRONQUIOS O DEL PULMÓN, P...",2,4845612.48
3257142,201902,5147267,TRATAMIENTO AMBULATORIO POR CANCER,C349,"TUMOR MALIGNO DE LOS BRONQUIOS O DEL PULMÓN, P...",1,26928477.36
3288952,201808,5147267,CONSULTA EXTERNA,9,DIAGNÓSTICO PENDIENTE,5,165168.00
3290884,201910,5147267,LABORATORIO CLINICO,9,DIAGNÓSTICO PENDIENTE,9,203450.52


In [ ]:
siniestros['Fecha_pago'] = pd.to_datetime(siniestros['Mes_Pago'], errors = 'coerce', format = '%Y%m')

In [ ]:
siniestros['Mes_Pago'] = pd.to_datetime(siniestros['Mes_Pago'], errors = 'coerce', format = '%Y%m')

In [ ]:
siniestros['Año_Pago'] = siniestros['Mes_Pago'].dt.year
siniestros['Mes_Pago'] = siniestros['Mes_Pago'].dt.month

In [ ]:
siniestros = siniestros.reindex(columns=['Fecha_pago','Año_Pago', 'Mes_Pago', 'Asegurado_Id',	'Reclamacion',	'Diagnostico_Codigo',	'Diagnostico_Desc',	'Eventos',	'Valor_Pagado'])


In [ ]:
#Cambiar el formato de los nombres de las columnas
siniestros.columns = siniestros.columns.str.capitalize()

In [ ]:
siniestros = siniestros[siniestros['Valor_pagado'] > 0]

In [ ]:
siniestros['Valor_pagado'] = siniestros['Valor_pagado'].apply(lambda x: 0 if (x < 0) else x)

In [ ]:
siniestros

,Fecha_pago,Año_pago,Mes_pago,Asegurado_id,Reclamacion,Diagnostico_codigo,Diagnostico_desc,Eventos,Valor_pagado
0,2018-03-01,2018,3,4834968,EXAMENES DE DIAGNOSTICO,Z011,EXAMEN DE OÍDOS Y DE LA AUDICIÓN,1,111600.00
1,2019-09-01,2019,9,14915125,EXAMENES DE DIAGNOSTICO,9,DIAGNÓSTICO PENDIENTE,1,50468.00
2,2018-07-01,2018,7,42959949,EXAMENES DE DIAGNOSTICO,9,DIAGNÓSTICO PENDIENTE,1,610452.00
3,2018-03-01,2018,3,30540131,CONSULTA EXTERNA,9,DIAGNÓSTICO PENDIENTE,1,42160.00
4,2019-05-01,2019,5,21442420,LABORATORIO CLINICO,9,DIAGNÓSTICO PENDIENTE,18,617336.48
...,...,...,...,...,...,...,...,...,...
3308475,2019-12-01,2019,12,6197783,EXAMENES DE DIAGNOSTICO,9,DIAGNÓSTICO PENDIENTE,1,90105.84
3308476,2019-05-01,2019,5,1001139,CONSULTA EXTERNA,9,DIAGNÓSTICO PENDIENTE,4,297352.00
3308477,2018-03-01,2018,3,21888100,CONSULTA DE URGENCIAS,9,DIAGNÓSTICO PENDIENTE,1,350354.56
3308478,2018-05-01,2018,5,56483856,CONSULTA DE URGENCIAS,9,DIAGNÓSTICO PENDIENTE,1,58280.00


In [ ]:
siniestros01 = siniestros[['Asegurado_id', 'Valor_pagado']]

In [ ]:
siniestros02 = siniestros01.groupby(['Asegurado_id'])[ 'Valor_pagado'].sum().reset_index()

In [ ]:
siniestros1 = pd.read_csv("/content/drive/Shareddrives/Analítica/NIVEL 3/Finanzas/SiniestrosFinal.csv")

In [ ]:
#Cambiar el formato de los nombres de las columnas
siniestros1.columns = siniestros1.columns.str.capitalize()

In [ ]:
df = pd.merge(siniestros1, siniestros02, how= "left")
df

,Unnamed: 0,Asegurado_id,Atencion del parto,Cirugia ambulatoria por accidente,Cirugia ambulatoria por enfermedad,Cobertura programa paliativos,Conciliacion,Consulta de urgencias,Consulta externa,Consultas prenatales,...,Tratamiento medico ambulatorio por accidentes,Tratamiento medico ambulatorio por enfermedad en urgencias,Tratamiento medico hospitalario por cancer y complicaciones,Tratamiento medico hospitalario por enfermedad,Tratamiento quirurgico hospitalario por accidente,Tratamiento quirurgico hospitalario por cancer y complicaciones,Tratamiento quirurgico hospitalario por enfermedad,Tratamientos ambulatorios especiales,Tratamientos especificos,Valor_pagado
0,0,738284,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1017112.48
1,1,738288,0,0,0,0,0,6,4,0,...,0,0,0,0,0,0,0,0,0,663763.32
2,2,738294,0,0,0,0,0,1,4,0,...,0,0,0,0,0,0,0,0,0,2831903.32
3,3,738303,0,0,0,0,0,5,6,0,...,1,0,0,0,0,0,0,0,0,5232771.48
4,4,738318,0,0,0,0,1,0,2,0,...,0,0,0,0,0,0,1,0,49,64044725.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237162,237162,58581352,3,0,0,0,0,3,13,7,...,0,0,0,2,0,0,0,0,0,44482031.44
237163,237163,58581939,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,442048.84
237164,237164,58581989,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,135319.96
237165,237165,58584529,0,0,0,0,0,1,3,0,...,0,0,0,0,0,0,0,0,0,423040.88


In [ ]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237167 entries, 0 to 237166
Data columns (total 28 columns):
 #   Column                                                           Non-Null Count   Dtype  
---  ------                                                           --------------   -----  
 0   Asegurado_id                                                     237167 non-null  int64  
 1   Atencion del parto                                               237167 non-null  int64  
 2   Cirugia ambulatoria por accidente                                237167 non-null  int64  
 3   Cirugia ambulatoria por enfermedad                               237167 non-null  int64  
 4   Cobertura programa paliativos                                    237167 non-null  int64  
 5   Conciliacion                                                     237167 non-null  int64  
 6   Consulta de urgencias                                            237167 non-null  int64  
 7   Consulta externa             

In [ ]:
df= df.dropna()

In [ ]:
siniestros1= df.astype({'Valor_pagado' : int})

In [ ]:
#Cambiar el formato de los nombres de las columnas
siniestros1.columns = siniestros1.columns.str.capitalize()

In [ ]:
siniestros1

,Asegurado_id,Atencion del parto,Cirugia ambulatoria por accidente,Cirugia ambulatoria por enfermedad,Cobertura programa paliativos,Conciliacion,Consulta de urgencias,Consulta externa,Consultas prenatales,Examenes de diagnostico,...,Tratamiento medico ambulatorio por accidentes,Tratamiento medico ambulatorio por enfermedad en urgencias,Tratamiento medico hospitalario por cancer y complicaciones,Tratamiento medico hospitalario por enfermedad,Tratamiento quirurgico hospitalario por accidente,Tratamiento quirurgico hospitalario por cancer y complicaciones,Tratamiento quirurgico hospitalario por enfermedad,Tratamientos ambulatorios especiales,Tratamientos especificos,Valor_pagado
0,738284,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1017112
1,738288,0,0,0,0,0,6,4,0,0,...,0,0,0,0,0,0,0,0,0,663763
2,738294,0,0,0,0,0,1,4,0,3,...,0,0,0,0,0,0,0,0,0,2831903
3,738303,0,0,0,0,0,5,6,0,8,...,1,0,0,0,0,0,0,0,0,5232771
4,738318,0,0,0,0,1,0,2,0,21,...,0,0,0,0,0,0,1,0,49,64044725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237162,58581352,3,0,0,0,0,3,13,7,11,...,0,0,0,2,0,0,0,0,0,44482031
237163,58581939,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,442048
237164,58581989,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,135319
237165,58584529,0,0,0,0,0,1,3,0,1,...,0,0,0,0,0,0,0,0,0,423040


In [ ]:
siniestros= siniestros.astype({'Valor_pagado' : int})

In [ ]:
siniestros

,Fecha_pago,Año_pago,Mes_pago,Asegurado_id,Reclamacion,Diagnostico_codigo,Diagnostico_desc,Eventos,Valor_pagado
0,2018-03-01,2018,3,4834968,EXAMENES DE DIAGNOSTICO,Z011,EXAMEN DE OÍDOS Y DE LA AUDICIÓN,1,111600
1,2019-09-01,2019,9,14915125,EXAMENES DE DIAGNOSTICO,9,DIAGNÓSTICO PENDIENTE,1,50468
2,2018-07-01,2018,7,42959949,EXAMENES DE DIAGNOSTICO,9,DIAGNÓSTICO PENDIENTE,1,610452
3,2018-03-01,2018,3,30540131,CONSULTA EXTERNA,9,DIAGNÓSTICO PENDIENTE,1,42160
4,2019-05-01,2019,5,21442420,LABORATORIO CLINICO,9,DIAGNÓSTICO PENDIENTE,18,617336
...,...,...,...,...,...,...,...,...,...
3308475,2019-12-01,2019,12,6197783,EXAMENES DE DIAGNOSTICO,9,DIAGNÓSTICO PENDIENTE,1,90105
3308476,2019-05-01,2019,5,1001139,CONSULTA EXTERNA,9,DIAGNÓSTICO PENDIENTE,4,297352
3308477,2018-03-01,2018,3,21888100,CONSULTA DE URGENCIAS,9,DIAGNÓSTICO PENDIENTE,1,350354
3308478,2018-05-01,2018,5,56483856,CONSULTA DE URGENCIAS,9,DIAGNÓSTICO PENDIENTE,1,58280



## Base de datos características sociodemográficas

In [ ]:
percent_null(sociodemograficas)

Asegurado_Id -> 0  |  0.0 %
Sexo_Cd -> 0  |  0.0 %
FechaNacimiento -> 0  |  0.0 %
Regional -> 1  |  0.0 %
CANCER -> 0  |  0.0 %
EPOC -> 0  |  0.0 %
DIABETES -> 0  |  0.0 %
HIPERTENSION -> 0  |  0.0 %
ENF_CARDIOVASCULAR -> 0  |  0.0 %


In [ ]:
sociodemograficas

,Asegurado_Id,Sexo_Cd,FechaNacimiento,Regional,CANCER,EPOC,DIABETES,HIPERTENSION,ENF_CARDIOVASCULAR
0,18686469,F,1984/01/17,REGIONAL ANTIOQUIA,0,0,0,0,0
1,16087775,M,1984/03/12,REGIONAL NORTE,0,0,0,0,0
2,35757761,F,1993/12/19,REGIONAL ANTIOQUIA,0,0,0,0,0
3,48691460,M,2000/11/14,REGIONAL ANTIOQUIA,0,0,0,0,0
4,36074099,F,2015/08/06,REGIONAL CENTRO,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
267307,21676160,F,1988/01/21,REGIONAL ANTIOQUIA,0,0,0,0,0
267308,4782725,F,1975/01/23,REGIONAL CENTRO,0,0,0,0,0
267309,10100593,F,1975/07/04,REGIONAL NORTE,1,0,0,0,0
267310,34178573,F,1978/10/15,REGIONAL CENTRO,0,0,0,0,0


In [ ]:
sociodemograficas[sociodemograficas['Asegurado_Id'] == 38260559]

,Asegurado_Id,Sexo_Cd,FechaNacimiento,Regional,CANCER,EPOC,DIABETES,HIPERTENSION,ENF_CARDIOVASCULAR
136719,38260559,M,2016/04/28,REGIONAL CENTRO,0,0,0,0,0
184068,38260559,M,2016/04/28,REGIONAL OCCIDENTE,0,0,0,0,0


In [ ]:
lista = sociodemograficas.columns
for i in lista:
    print(i)
    print(sociodemograficas[i].value_counts())
    print('__________________________')

Asegurado_Id
38260559    2
44419136    2
43387431    2
36757489    2
41624089    2
           ..
41266249    1
2195439     1
1819050     1
44244907    1
4419287     1
Name: Asegurado_Id, Length: 267233, dtype: int64
__________________________
Sexo_Cd
F     145430
M     121877
-1         5
Name: Sexo_Cd, dtype: int64
__________________________
FechaNacimiento
1985/01/25    42
2016/01/04    36
1979/10/28    34
1982/10/03    33
1980/10/23    33
              ..
1948/02/27     1
1933/05/13     1
1921/01/05     1
1928/10/03     1
1946/08/10     1
Name: FechaNacimiento, Length: 29945, dtype: int64
__________________________
Regional
REGIONAL CENTRO          127149
REGIONAL ANTIOQUIA        49722
REGIONAL OCCIDENTE        46180
REGIONAL NORTE            34845
REGIONAL EJE CAFETERO      9342
Sin Información              73
Name: Regional, dtype: int64
__________________________
CANCER
0    262929
1      4383
Name: CANCER, dtype: int64
__________________________
EPOC
0    266055
1      1257
Nam

In [ ]:
sociodemograficas['Asegurado_Id'] = sociodemograficas['Asegurado_Id'].drop_duplicates()

In [ ]:
sociodemograficas.dropna()

,Asegurado_Id,Sexo_Cd,FechaNacimiento,Regional,CANCER,EPOC,DIABETES,HIPERTENSION,ENF_CARDIOVASCULAR
0,18686469.0,F,1984/01/17,REGIONAL ANTIOQUIA,0,0,0,0,0
1,16087775.0,M,1984/03/12,REGIONAL NORTE,0,0,0,0,0
2,35757761.0,F,1993/12/19,REGIONAL ANTIOQUIA,0,0,0,0,0
3,48691460.0,M,2000/11/14,REGIONAL ANTIOQUIA,0,0,0,0,0
4,36074099.0,F,2015/08/06,REGIONAL CENTRO,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
267307,21676160.0,F,1988/01/21,REGIONAL ANTIOQUIA,0,0,0,0,0
267308,4782725.0,F,1975/01/23,REGIONAL CENTRO,0,0,0,0,0
267309,10100593.0,F,1975/07/04,REGIONAL NORTE,1,0,0,0,0
267310,34178573.0,F,1978/10/15,REGIONAL CENTRO,0,0,0,0,0


Se reemplaza por los valores más frecuentes de cada variable

In [ ]:
sociodemograficas['Regional'] = sociodemograficas['Regional'].replace('Sin Información', 'REGIONAL CENTRO')
sociodemograficas['Sexo_Cd'] = sociodemograficas['Sexo_Cd'].apply(lambda x: 0 if x == "F" else 1 if x == "M" else 0)


In [ ]:
sociodemograficas.dtypes

Asegurado_Id          float64
Sexo_Cd                 int64
FechaNacimiento        object
Regional               object
CANCER                  int64
EPOC                    int64
DIABETES                int64
HIPERTENSION            int64
ENF_CARDIOVASCULAR      int64
dtype: object

In [ ]:
sociodemograficas['FechaNacimiento'] = pd.to_datetime(sociodemograficas['FechaNacimiento'], errors = 'coerce', format = '%Y-%m-%d')

In [ ]:
sociodemograficas= sociodemograficas.rename(columns={'FechaNacimiento':'Fecha_Nacimiento'})

In [ ]:
#Cambiar el formato de los nombres de las columnas
sociodemograficas.columns = sociodemograficas.columns.str.capitalize()

In [ ]:
sociodemograficas['Edad'] = 2019- sociodemograficas['Fecha_nacimiento'].dt.year

In [ ]:
sociodemograficas[sociodemograficas['Edad'] < 0]

,Asegurado_id,Sexo_cd,Fecha_nacimiento,Regional,Cancer,Epoc,Diabetes,Hipertension,Enf_cardiovascular,Edad
3226,51687029.0,1,2020-01-03,REGIONAL NORTE,0,0,0,0,0,-1
4387,51631367.0,0,2020-01-14,REGIONAL EJE CAFETERO,0,0,0,0,0,-1
4865,51555107.0,1,2020-01-12,REGIONAL CENTRO,0,0,0,0,0,-1
6967,51501797.0,1,2020-01-04,REGIONAL CENTRO,0,0,0,0,0,-1
7074,52138380.0,0,2020-01-18,REGIONAL OCCIDENTE,0,0,0,0,0,-1
...,...,...,...,...,...,...,...,...,...,...
262154,51534980.0,1,2020-01-05,REGIONAL ANTIOQUIA,0,0,0,0,0,-1
262290,36108106.0,0,2022-02-12,REGIONAL CENTRO,0,0,0,0,0,-3
264521,51700175.0,1,2020-01-14,REGIONAL CENTRO,0,0,0,0,0,-1
265418,51568459.0,1,2020-01-11,REGIONAL OCCIDENTE,0,0,0,0,0,-1


In [ ]:
sociodemograficas.sort_values(['Edad'])

,Asegurado_id,Sexo_cd,Fecha_nacimiento,Regional,Cancer,Epoc,Diabetes,Hipertension,Enf_cardiovascular,Edad
169662,56691496.0,0,2023-09-29,REGIONAL OCCIDENTE,0,0,0,0,0,-4
64566,44278989.0,0,2023-09-29,REGIONAL CENTRO,0,0,0,0,0,-4
257247,57482107.0,0,2023-09-29,REGIONAL CENTRO,0,0,0,0,0,-4
64352,23768169.0,0,2023-09-29,REGIONAL CENTRO,0,0,0,0,0,-4
105306,58197213.0,1,2022-03-05,REGIONAL CENTRO,0,0,0,0,0,-3
...,...,...,...,...,...,...,...,...,...,...
125733,11852857.0,0,1917-06-15,REGIONAL NORTE,1,0,0,0,0,102
71868,12029496.0,0,1917-10-02,REGIONAL CENTRO,0,0,0,0,0,102
115378,6069815.0,0,1916-04-28,REGIONAL CENTRO,0,0,0,0,0,103
234997,2805068.0,0,1916-01-24,REGIONAL ANTIOQUIA,0,0,0,0,0,103


In [ ]:
sociodemograficas[sociodemograficas['Edad'] < 0]

,Asegurado_id,Sexo_cd,Fecha_nacimiento,Regional,Cancer,Epoc,Diabetes,Hipertension,Enf_cardiovascular,Edad
3226,51687029.0,1,2020-01-03,REGIONAL NORTE,0,0,0,0,0,-1
4387,51631367.0,0,2020-01-14,REGIONAL EJE CAFETERO,0,0,0,0,0,-1
4865,51555107.0,1,2020-01-12,REGIONAL CENTRO,0,0,0,0,0,-1
6967,51501797.0,1,2020-01-04,REGIONAL CENTRO,0,0,0,0,0,-1
7074,52138380.0,0,2020-01-18,REGIONAL OCCIDENTE,0,0,0,0,0,-1
...,...,...,...,...,...,...,...,...,...,...
262154,51534980.0,1,2020-01-05,REGIONAL ANTIOQUIA,0,0,0,0,0,-1
262290,36108106.0,0,2022-02-12,REGIONAL CENTRO,0,0,0,0,0,-3
264521,51700175.0,1,2020-01-14,REGIONAL CENTRO,0,0,0,0,0,-1
265418,51568459.0,1,2020-01-11,REGIONAL OCCIDENTE,0,0,0,0,0,-1


In [ ]:
sociodemograficas['Edad'] = sociodemograficas['Edad'].apply(lambda x: 0 if x < 0 else  x)

In [ ]:
sociodemograficas.drop(['Fecha_nacimiento'], axis = 1, inplace = True)

In [ ]:
sociodemograficas = sociodemograficas.dropna()

In [ ]:
sociodemograficas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267232 entries, 0 to 267311
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Asegurado_id        267232 non-null  float64
 1   Sexo_cd             267232 non-null  int64  
 2   Regional            267232 non-null  object 
 3   Cancer              267232 non-null  int64  
 4   Epoc                267232 non-null  int64  
 5   Diabetes            267232 non-null  int64  
 6   Hipertension        267232 non-null  int64  
 7   Enf_cardiovascular  267232 non-null  int64  
 8   Edad                267232 non-null  int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 20.4+ MB


In [ ]:
sociodemograficas = sociodemograficas.astype({'Asegurado_id' : int})

In [ ]:
sociodemograficas[sociodemograficas.duplicated()]

,Asegurado_id,Sexo_cd,Regional,Cancer,Epoc,Diabetes,Hipertension,Enf_cardiovascular,Edad


Se realiza la agrupación de las edades: está segmentación de edades se hace en base a la clasificación de edades hecha por Adwords: https://adwords-ppc.es/segmentacion-por-edad-y-sexo-en-adwords/

de 0 a 17 años.

de 18 a 24 años.

de 25 a 34 años.

de 35 a 44 años.

de 45 a 54 años.

de 55 a 64 años.

de 65 o mayor.




In [ ]:
sociodemograficas['Edad'] = sociodemograficas['Edad'].apply(lambda x: "0-17" if ((x >= 0) & (x <= 17)) else "18-24" if ((x >= 18) & (x <= 24)) else "25-34" if ((x >= 25) & (x <= 34)) else  "35-44" if ((x >= 35) & (x <= 44)) else "45-54" if ((x >= 45) & (x <= 54)) else "55-64" if ((x >= 55) & (x <= 64)) else "65 o mayor" )

In [ ]:
sociodemograficas['Edad'].value_counts()

0-17          70590
35-44         59512
25-34         50807
45-54         32817
55-64         21830
18-24         19009
65 o mayor    12667
Name: Edad, dtype: int64

## Base de datos expuestos

In [ ]:
percent_null(expuestos)

Asegurado_Id -> 0  |  0.0 %
Poliza_Asegurado_Id -> 0  |  0.0 %
FECHA_INICIO -> 0  |  0.0 %
FECHA_FIN -> 0  |  0.0 %
expo  -> 0  |  0.0 %


In [ ]:
expuestos

,Asegurado_Id,Poliza_Asegurado_Id,FECHA_INICIO,FECHA_FIN,expo
0,738284,89612446,1/1/2018,2/1/2018,0.04
1,738288,88146604,1/1/2018,11/8/2018,0.43
2,738288,175992661,11/8/2018,12/31/2019,0.57
3,738294,89612649,1/1/2018,12/31/2019,1.00
4,738303,120708179,1/1/2018,12/31/2019,1.00
...,...,...,...,...,...
300895,58581939,168944085,5/15/2018,12/31/2019,0.82
300896,58581989,132872337,1/1/2018,12/31/2019,1.00
300897,58584529,96109296,1/1/2018,9/1/2018,0.33
300898,58584580,159465186,1/1/2018,12/31/2019,1.00


In [ ]:
lista = expuestos.columns
for i in lista:
    print(i)
    print(expuestos[i].value_counts())
    print('__________________________')

Asegurado_Id
43841424    7
13175696    6
30787886    6
34629992    6
41391506    5
           ..
15529715    1
15529731    1
15529882    1
15530130    1
58584620    1
Name: Asegurado_Id, Length: 267233, dtype: int64
__________________________
Poliza_Asegurado_Id
89612446     1
203532076    1
193485530    1
158481541    1
89813000     1
            ..
162926960    1
107445845    1
126795083    1
137557576    1
94495059     1
Name: Poliza_Asegurado_Id, Length: 300900, dtype: int64
__________________________
FECHA_INICIO
1/1/2018      191371
12/1/2018       2139
2/1/2018        1830
6/1/2019        1591
3/1/2019        1579
               ...  
11/3/2019         14
6/3/2019          14
6/23/2019         12
1/14/2018         10
11/18/2018         7
Name: FECHA_INICIO, Length: 730, dtype: int64
__________________________
FECHA_FIN
12/31/2019    217004
12/1/2018       1967
7/1/2019        1499
2/1/2018        1455
6/1/2018        1417
               ...  
11/11/2019        25
6/9/2018       

In [ ]:
expuestos.dtypes

Asegurado_Id             int64
Poliza_Asegurado_Id      int64
FECHA_INICIO            object
FECHA_FIN               object
expo                   float64
dtype: object

In [ ]:
expuestos['FECHA_INICIO'] = pd.to_datetime(expuestos['FECHA_INICIO'])
expuestos['FECHA_FIN'] = pd.to_datetime(expuestos['FECHA_FIN'])

In [ ]:
expuestos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300900 entries, 0 to 300899
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Asegurado_Id         300900 non-null  int64         
 1   Poliza_Asegurado_Id  300900 non-null  int64         
 2   FECHA_INICIO         300900 non-null  datetime64[ns]
 3   FECHA_FIN            300900 non-null  datetime64[ns]
 4   expo                 300900 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(2)
memory usage: 11.5 MB


In [ ]:
expuestos.columns = expuestos.columns.str.capitalize()

In [ ]:
expuestos[expuestos['Asegurado_id'] == 43841424]

,Asegurado_id,Poliza_asegurado_id,Fecha_inicio,Fecha_fin,Expo
251638,43841424,126126467,2018-01-01,2018-04-01,0.12
251639,43841424,126097796,2018-01-01,2018-04-01,0.12
251640,43841424,168027384,2018-04-01,2018-09-01,0.21
251641,43841424,166825611,2018-04-01,2018-04-01,0.00
251642,43841424,174630133,2018-09-01,2019-12-31,0.67
251643,43841424,172413620,2018-09-01,2018-12-01,0.12
251644,43841424,177890562,2018-12-01,2019-12-31,0.54


In [ ]:
"""
df1 = expuestos.copy(deep = True)
lista_usuarios = list(expuestos['Asegurado_id'].unique())
df2 = pd.DataFrame()
for i in lista_usuarios:
    filtro = df1[df1['Asegurado_id']== i].reset_index()
    if filtro.shape[0] > 1: #usuarios que tengan más de 1 registro
        min = filtro['Fecha_inicio'].min()
        max = filtro['Fecha_fin'].max()
        nueva_fila = {'Asegurado_id': i, 'Fecha_inicio':min, 'Fecha_fin':max}
    else: 
        nueva_fila = filtro
    df2 = df2.append(nueva_fila, ignore_index= True)

"""

"\ndf1 = expuestos.copy(deep = True)\nlista_usuarios = list(expuestos['Asegurado_id'].unique())\ndf2 = pd.DataFrame()\nfor i in lista_usuarios:\n    filtro = df1[df1['Asegurado_id']== i].reset_index()\n    if filtro.shape[0] > 1: #usuarios que tengan más de 1 registro\n        min = filtro['Fecha_inicio'].min()\n        max = filtro['Fecha_fin'].max()\n        nueva_fila = {'Asegurado_id': i, 'Fecha_inicio':min, 'Fecha_fin':max}\n    else: \n        nueva_fila = filtro\n    df2 = df2.append(nueva_fila, ignore_index= True)\n\n"

In [ ]:
#df1 = expuestos.copy(deep = True)

In [ ]:
"""
lista_usuarios = list(expuestos['Asegurado_id'][expuestos['Asegurado_id'].duplicated()].unique())
df2 = pd.DataFrame()
for i in lista_usuarios:
    filtro = df1[df1['Asegurado_id']== i].reset_index()
    if filtro.shape[0] > 1: #usuarios que tengan más de 1 registro
        min = filtro['Fecha_inicio'].min()
        max = filtro['Fecha_fin'].max()
        nueva_fila = {'Asegurado_id': i, 'Fecha_inicio':min, 'Fecha_fin':max}
    df2 = df2.append(nueva_fila, ignore_index= True)
"""

"\nlista_usuarios = list(expuestos['Asegurado_id'][expuestos['Asegurado_id'].duplicated()].unique())\ndf2 = pd.DataFrame()\nfor i in lista_usuarios:\n    filtro = df1[df1['Asegurado_id']== i].reset_index()\n    if filtro.shape[0] > 1: #usuarios que tengan más de 1 registro\n        min = filtro['Fecha_inicio'].min()\n        max = filtro['Fecha_fin'].max()\n        nueva_fila = {'Asegurado_id': i, 'Fecha_inicio':min, 'Fecha_fin':max}\n    df2 = df2.append(nueva_fila, ignore_index= True)\n"

In [ ]:
#df2

In [ ]:
#df3 = expuestos[~ expuestos['Asegurado_id'].duplicated()]
#df3 = df3.drop(['Expo ','Poliza_asegurado_id'], axis=1)
#expuestos1 = pd.concat([df2, df3]).reset_index().drop('index', axis=1)

In [ ]:
#expuestos1[expuestos1.duplicated()]

In [ ]:
#df2 = df2.drop(['Expo ','Poliza_asegurado_id', 'index'], axis=1)

In [ ]:
#from google.colab import files

In [ ]:
#df2.to_csv('/content/drive/Shareddrives/Analítica/NIVEL 3/Finanzas/expuestosF.csv', index = False)
#files.download('/content/drive/Shareddrives/Analítica/NIVEL 3/Finanzas/expuestosF.csv') 

In [ ]:
expuestos = pd.read_csv("/content/drive/Shareddrives/Analítica/NIVEL 3/Finanzas/expuestosF1.csv")

In [ ]:
expuestos

,Asegurado_id,Fecha_inicio,Fecha_fin,Expo
0,738284,1/1/2018,2/1/2018,0.04
1,738288,1/1/2018,12/31/2019,1.00
2,738294,1/1/2018,12/31/2019,1.00
3,738303,1/1/2018,12/31/2019,1.00
4,738318,1/1/2018,12/31/2019,1.00
...,...,...,...,...
267228,58581939,5/9/2018,12/31/2019,0.82
267229,58581989,1/1/2018,12/31/2019,1.00
267230,58584529,1/1/2018,9/1/2018,0.33
267231,58584580,1/1/2018,12/31/2019,1.00


In [ ]:
expuestos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267233 entries, 0 to 267232
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Asegurado_id  267233 non-null  int64  
 1   Fecha_inicio  267233 non-null  object 
 2   Fecha_fin     267233 non-null  object 
 3   Expo          267233 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 8.2+ MB


In [ ]:
expuestos= expuestos.rename(columns={'Expo ':'Expo'})

In [ ]:
expuestos[expuestos['Expo'] < 0]

,Asegurado_id,Fecha_inicio,Fecha_fin,Expo
254959,51346104,11/26/2019,4/2/2019,-0.33


In [ ]:
expuestos = expuestos[expuestos['Expo'] > 0]

In [ ]:
expuestos

,Asegurado_id,Fecha_inicio,Fecha_fin,Expo
0,738284,1/1/2018,2/1/2018,0.04
1,738288,1/1/2018,12/31/2019,1.00
2,738294,1/1/2018,12/31/2019,1.00
3,738303,1/1/2018,12/31/2019,1.00
4,738318,1/1/2018,12/31/2019,1.00
...,...,...,...,...
267228,58581939,5/9/2018,12/31/2019,0.82
267229,58581989,1/1/2018,12/31/2019,1.00
267230,58584529,1/1/2018,9/1/2018,0.33
267231,58584580,1/1/2018,12/31/2019,1.00


In [ ]:
expuestos

,Asegurado_id,Fecha_inicio,Fecha_fin,Expo
0,738284,1/1/2018,2/1/2018,0.04
1,738288,1/1/2018,12/31/2019,1.00
2,738294,1/1/2018,12/31/2019,1.00
3,738303,1/1/2018,12/31/2019,1.00
4,738318,1/1/2018,12/31/2019,1.00
...,...,...,...,...
267228,58581939,5/9/2018,12/31/2019,0.82
267229,58581989,1/1/2018,12/31/2019,1.00
267230,58584529,1/1/2018,9/1/2018,0.33
267231,58584580,1/1/2018,12/31/2019,1.00


#**Bodega de datos**

In [ ]:
df = pd.merge(sociodemograficas, expuestos, how = 'outer')

In [ ]:
df.dropna()

,Asegurado_id,Sexo_cd,Regional,Cancer,Epoc,Diabetes,Hipertension,Enf_cardiovascular,Edad,Fecha_inicio,Fecha_fin,Expo
0,18686469,0.0,REGIONAL ANTIOQUIA,0.0,0.0,0.0,0.0,0.0,35-44,10/31/2019,12/31/2019,0.08
1,16087775,1.0,REGIONAL NORTE,0.0,0.0,0.0,0.0,0.0,35-44,1/1/2018,12/31/2019,1.00
2,35757761,0.0,REGIONAL ANTIOQUIA,0.0,0.0,0.0,0.0,0.0,25-34,1/1/2018,12/31/2019,1.00
3,48691460,1.0,REGIONAL ANTIOQUIA,0.0,0.0,0.0,0.0,0.0,18-24,1/1/2018,12/31/2019,1.00
4,36074099,0.0,REGIONAL CENTRO,0.0,0.0,0.0,0.0,0.0,0-17,1/1/2018,12/31/2019,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...
267227,21676160,0.0,REGIONAL ANTIOQUIA,0.0,0.0,0.0,0.0,0.0,25-34,1/1/2018,12/31/2019,1.00
267228,4782725,0.0,REGIONAL CENTRO,0.0,0.0,0.0,0.0,0.0,35-44,1/1/2018,12/31/2019,1.00
267229,10100593,0.0,REGIONAL NORTE,1.0,0.0,0.0,0.0,0.0,35-44,1/1/2018,12/31/2019,1.00
267230,34178573,0.0,REGIONAL CENTRO,0.0,0.0,0.0,0.0,0.0,35-44,1/1/2018,12/31/2019,1.00


In [ ]:
BF = pd.merge(siniestros1, df)

In [ ]:
BF

,Asegurado_id,Atencion del parto,Cirugia ambulatoria por accidente,Cirugia ambulatoria por enfermedad,Cobertura programa paliativos,Conciliacion,Consulta de urgencias,Consulta externa,Consultas prenatales,Examenes de diagnostico,...,Regional,Cancer,Epoc,Diabetes,Hipertension,Enf_cardiovascular,Edad,Fecha_inicio,Fecha_fin,Expo
0,738284,0,0,0,0,0,0,1,0,1,...,REGIONAL CENTRO,0.0,0.0,0.0,0.0,0.0,45-54,1/1/2018,2/1/2018,0.04
1,738288,0,0,0,0,0,6,4,0,0,...,REGIONAL ANTIOQUIA,0.0,0.0,0.0,0.0,0.0,35-44,1/1/2018,12/31/2019,1.00
2,738294,0,0,0,0,0,1,4,0,3,...,REGIONAL CENTRO,0.0,0.0,0.0,0.0,0.0,55-64,1/1/2018,12/31/2019,1.00
3,738303,0,0,0,0,0,5,6,0,8,...,REGIONAL NORTE,0.0,0.0,0.0,0.0,0.0,45-54,1/1/2018,12/31/2019,1.00
4,738318,0,0,0,0,1,0,2,0,21,...,REGIONAL CENTRO,1.0,0.0,1.0,1.0,0.0,65 o mayor,1/1/2018,12/31/2019,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235440,58581352,3,0,0,0,0,3,13,7,11,...,REGIONAL CENTRO,0.0,0.0,0.0,0.0,0.0,35-44,1/1/2018,12/31/2019,1.00
235441,58581939,0,0,0,0,0,0,2,0,0,...,REGIONAL ANTIOQUIA,0.0,0.0,0.0,0.0,0.0,35-44,5/9/2018,12/31/2019,0.82
235442,58581989,0,0,0,0,0,0,0,0,1,...,REGIONAL ANTIOQUIA,0.0,0.0,0.0,0.0,0.0,0-17,1/1/2018,12/31/2019,1.00
235443,58584529,0,0,0,0,0,1,3,0,1,...,REGIONAL ANTIOQUIA,0.0,0.0,0.0,0.0,0.0,0-17,1/1/2018,9/1/2018,0.33


In [ ]:
percent_null(BF)

Asegurado_id -> 0  |  0.0 %
Atencion del parto -> 0  |  0.0 %
Cirugia ambulatoria por accidente -> 0  |  0.0 %
Cirugia ambulatoria por enfermedad -> 0  |  0.0 %
Cobertura programa paliativos -> 0  |  0.0 %
Conciliacion -> 0  |  0.0 %
Consulta de urgencias -> 0  |  0.0 %
Consulta externa -> 0  |  0.0 %
Consultas prenatales -> 0  |  0.0 %
Examenes de diagnostico -> 0  |  0.0 %
Fisioterapia ilimitada -> 0  |  0.0 %
Infiltraciones -> 0  |  0.0 %
Laboratorio clinico -> 0  |  0.0 %
Odontología -> 0  |  0.0 %
Otros -> 0  |  0.0 %
Pamd -> 0  |  0.0 %
Traslado en ambulancia -> 0  |  0.0 %
Tratamiento ambulatorio por cancer -> 0  |  0.0 %
Tratamiento medico ambulatorio por accidentes -> 0  |  0.0 %
Tratamiento medico ambulatorio por enfermedad en urgencias -> 0  |  0.0 %
Tratamiento medico hospitalario por cancer y complicaciones -> 0  |  0.0 %
Tratamiento medico hospitalario por enfermedad -> 0  |  0.0 %
Tratamiento quirurgico hospitalario por accidente -> 0  |  0.0 %
Tratamiento quirurgico hos

In [ ]:
bodegaF = BF.dropna()

In [ ]:
bodegaF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234877 entries, 0 to 235444
Data columns (total 39 columns):
 #   Column                                                           Non-Null Count   Dtype  
---  ------                                                           --------------   -----  
 0   Asegurado_id                                                     234877 non-null  int64  
 1   Atencion del parto                                               234877 non-null  int64  
 2   Cirugia ambulatoria por accidente                                234877 non-null  int64  
 3   Cirugia ambulatoria por enfermedad                               234877 non-null  int64  
 4   Cobertura programa paliativos                                    234877 non-null  int64  
 5   Conciliacion                                                     234877 non-null  int64  
 6   Consulta de urgencias                                            234877 non-null  int64  
 7   Consulta externa             

In [ ]:
bodega = bodegaF.drop(['Fecha_inicio','Fecha_fin'], axis=1)

In [ ]:
bodega  = bodega.reset_index().drop('index', axis=1)

###Bodega final

In [ ]:
bodegaf = bodega.astype({'Cancer' : int, 'Epoc' : int, 'Enf_cardiovascular': int, 'Diabetes': int, 'Hipertension': int, 'Sexo_cd': int})

In [ ]:
bodegaf.columns

Index(['Asegurado_id', 'Atencion del parto',
       'Cirugia ambulatoria por accidente',
       'Cirugia ambulatoria por enfermedad', 'Cobertura programa paliativos',
       'Conciliacion', 'Consulta de urgencias', 'Consulta externa',
       'Consultas prenatales', 'Examenes de diagnostico',
       'Fisioterapia ilimitada', 'Infiltraciones', 'Laboratorio clinico',
       'Odontología', 'Otros', 'Pamd', 'Traslado en ambulancia',
       'Tratamiento ambulatorio por cancer',
       'Tratamiento medico ambulatorio por accidentes',
       'Tratamiento medico ambulatorio por enfermedad en urgencias',
       'Tratamiento medico hospitalario por cancer y complicaciones',
       'Tratamiento medico hospitalario por enfermedad',
       'Tratamiento quirurgico hospitalario por accidente',
       'Tratamiento quirurgico hospitalario por cancer y complicaciones',
       'Tratamiento quirurgico hospitalario por enfermedad',
       'Tratamientos ambulatorios especiales', 'Tratamientos especificos',
 

In [ ]:
df_num= bodegaf[['Atencion del parto',
       'Cirugia ambulatoria por accidente',
       'Cirugia ambulatoria por enfermedad', 'Cobertura programa paliativos',
       'Conciliacion', 'Consulta de urgencias', 'Consulta externa',
       'Consultas prenatales', 'Examenes de diagnostico',
       'Fisioterapia ilimitada', 'Infiltraciones', 'Laboratorio clinico',
       'Odontología', 'Otros', 'Pamd', 'Traslado en ambulancia',
       'Tratamiento ambulatorio por cancer',
       'Tratamiento medico ambulatorio por accidentes',
       'Tratamiento medico ambulatorio por enfermedad en urgencias',
       'Tratamiento medico hospitalario por cancer y complicaciones',
       'Tratamiento medico hospitalario por enfermedad',
       'Tratamiento quirurgico hospitalario por accidente',
       'Tratamiento quirurgico hospitalario por cancer y complicaciones',
       'Tratamiento quirurgico hospitalario por enfermedad',
       'Tratamientos ambulatorios especiales', 'Tratamientos especificos',
        'Sexo_cd', 'Cancer', 'Epoc', 'Diabetes',
       'Hipertension', 'Enf_cardiovascular',  'Expo']]

In [ ]:
id = bodegaf[['Asegurado_id']]

In [ ]:
df_cat_nom = bodegaf[['Regional']]

In [ ]:
df_cat_ord = bodegaf[['Edad']]

In [ ]:
'''
df_cat_nom['Regional'] =df_cat_nom['Regional'].replace('REGIONAL CENTRO', 1 )
df_cat_nom['Regional'] =df_cat_nom['Regional'].replace('REGIONAL ANTIOQUIA', 2 )
df_cat_nom['Regional'] =df_cat_nom['Regional'].replace('REGIONAL OCCIDENTE', 3 )
df_cat_nom['Regional'] =df_cat_nom['Regional'].replace('REGIONAL NORTE', 4 )
df_cat_nom['Regional'] =df_cat_nom['Regional'].replace('REGIONAL EJE CAFETERO', 5)

'''

"\ndf_cat_nom['Regional'] =df_cat_nom['Regional'].replace('REGIONAL CENTRO', 1 )\ndf_cat_nom['Regional'] =df_cat_nom['Regional'].replace('REGIONAL ANTIOQUIA', 2 )\ndf_cat_nom['Regional'] =df_cat_nom['Regional'].replace('REGIONAL OCCIDENTE', 3 )\ndf_cat_nom['Regional'] =df_cat_nom['Regional'].replace('REGIONAL NORTE', 4 )\ndf_cat_nom['Regional'] =df_cat_nom['Regional'].replace('REGIONAL EJE CAFETERO', 5)\n\n"

In [ ]:
df_cat_ord['Edad'] = df_cat_ord['Edad'].replace('0-17', 1 )
df_cat_ord['Edad'] = df_cat_ord['Edad'].replace('18-24', 2)
df_cat_ord['Edad'] = df_cat_ord['Edad'].replace('25-34', 3 )
df_cat_ord['Edad'] = df_cat_ord['Edad'].replace('35-44', 4 )
df_cat_ord['Edad'] = df_cat_ord['Edad'].replace('45-54', 5 )
df_cat_ord['Edad'] = df_cat_ord['Edad'].replace('55-64', 6 )
df_cat_ord['Edad'] = df_cat_ord['Edad'].replace('65 o mayor', 7 )


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
df_dummies=pd.get_dummies(df_cat_nom).reset_index().drop('index', axis=1)

In [ ]:
df_dummies

,Regional_REGIONAL ANTIOQUIA,Regional_REGIONAL CENTRO,Regional_REGIONAL EJE CAFETERO,Regional_REGIONAL NORTE,Regional_REGIONAL OCCIDENTE
0,0,1,0,0,0
1,1,0,0,0,0
2,0,1,0,0,0
3,0,0,0,1,0
4,0,1,0,0,0
...,...,...,...,...,...
234872,0,1,0,0,0
234873,1,0,0,0,0
234874,1,0,0,0,0
234875,1,0,0,0,0


In [ ]:
df_cat_ord =df_cat_ord.reset_index().drop('index', axis=1)

In [ ]:
bodega_final0 = pd.concat([df_cat_ord, df_dummies], axis = 1)
bodega_final0

,Edad,Regional_REGIONAL ANTIOQUIA,Regional_REGIONAL CENTRO,Regional_REGIONAL EJE CAFETERO,Regional_REGIONAL NORTE,Regional_REGIONAL OCCIDENTE
0,5,0,1,0,0,0
1,4,1,0,0,0,0
2,6,0,1,0,0,0
3,5,0,0,0,1,0
4,7,0,1,0,0,0
...,...,...,...,...,...,...
234872,4,0,1,0,0,0
234873,4,1,0,0,0,0
234874,1,1,0,0,0,0
234875,1,1,0,0,0,0


In [ ]:
#Aplicar método de Reescalado a las variables numéricas
arreglo = df_num.values     

scalador = MinMaxScaler(feature_range=(0,1))
dReescalados = scalador.fit_transform(arreglo)
np.set_printoptions(precision=2, suppress=True)
print(dReescalados)

[[0.   0.   0.   ... 0.   0.   0.03]
 [0.   0.   0.   ... 0.   0.   1.  ]
 [0.   0.   0.   ... 0.   0.   1.  ]
 ...
 [0.   0.   0.   ... 0.   0.   1.  ]
 [0.   0.   0.   ... 0.   0.   0.32]
 [0.   0.   0.   ... 0.   0.   1.  ]]


In [ ]:
list1 = dReescalados.tolist() 
df_R = pd.DataFrame(list1) #convertir a DataFrame
lista = df_num.columns
for i in range(len(lista)):
    df_R = df_R.rename(columns= {i : lista[i]}) 
df_R

,Atencion del parto,Cirugia ambulatoria por accidente,Cirugia ambulatoria por enfermedad,Cobertura programa paliativos,Conciliacion,Consulta de urgencias,Consulta externa,Consultas prenatales,Examenes de diagnostico,Fisioterapia ilimitada,...,Tratamiento quirurgico hospitalario por enfermedad,Tratamientos ambulatorios especiales,Tratamientos especificos,Sexo_cd,Cancer,Epoc,Diabetes,Hipertension,Enf_cardiovascular,Expo
0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.002571,0.000000,0.006579,0.001043,...,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.030303
1,0.0,0.0,0.0,0.0,0.000000,0.052174,0.010283,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.000000
2,0.0,0.0,0.0,0.0,0.000000,0.008696,0.010283,0.000000,0.019737,0.000000,...,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.000000
3,0.0,0.0,0.0,0.0,0.000000,0.043478,0.015424,0.000000,0.052632,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
4,0.0,0.0,0.0,0.0,0.333333,0.000000,0.005141,0.000000,0.138158,0.000000,...,0.066667,0.0,0.441441,1.0,1.0,0.0,1.0,1.0,0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234872,0.6,0.0,0.0,0.0,0.000000,0.026087,0.033419,0.333333,0.072368,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
234873,0.0,0.0,0.0,0.0,0.000000,0.000000,0.005141,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.818182
234874,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.006579,0.000000,...,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.000000
234875,0.0,0.0,0.0,0.0,0.000000,0.008696,0.007712,0.000000,0.006579,0.000000,...,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.323232


In [ ]:
X = pd.concat([ df_R,bodega_final0 ],  axis = 1)
X

,Atencion del parto,Cirugia ambulatoria por accidente,Cirugia ambulatoria por enfermedad,Cobertura programa paliativos,Conciliacion,Consulta de urgencias,Consulta externa,Consultas prenatales,Examenes de diagnostico,Fisioterapia ilimitada,...,Diabetes,Hipertension,Enf_cardiovascular,Expo,Edad,Regional_REGIONAL ANTIOQUIA,Regional_REGIONAL CENTRO,Regional_REGIONAL EJE CAFETERO,Regional_REGIONAL NORTE,Regional_REGIONAL OCCIDENTE
0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.002571,0.000000,0.006579,0.001043,...,0.0,0.0,0.0,0.030303,5,0,1,0,0,0
1,0.0,0.0,0.0,0.0,0.000000,0.052174,0.010283,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,1.000000,4,1,0,0,0,0
2,0.0,0.0,0.0,0.0,0.000000,0.008696,0.010283,0.000000,0.019737,0.000000,...,0.0,0.0,0.0,1.000000,6,0,1,0,0,0
3,0.0,0.0,0.0,0.0,0.000000,0.043478,0.015424,0.000000,0.052632,0.000000,...,0.0,0.0,0.0,1.000000,5,0,0,0,1,0
4,0.0,0.0,0.0,0.0,0.333333,0.000000,0.005141,0.000000,0.138158,0.000000,...,1.0,1.0,0.0,1.000000,7,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234872,0.6,0.0,0.0,0.0,0.000000,0.026087,0.033419,0.333333,0.072368,0.000000,...,0.0,0.0,0.0,1.000000,4,0,1,0,0,0
234873,0.0,0.0,0.0,0.0,0.000000,0.000000,0.005141,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.818182,4,1,0,0,0,0
234874,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.006579,0.000000,...,0.0,0.0,0.0,1.000000,1,1,0,0,0,0
234875,0.0,0.0,0.0,0.0,0.000000,0.008696,0.007712,0.000000,0.006579,0.000000,...,0.0,0.0,0.0,0.323232,1,1,0,0,0,0


In [ ]:
y = bodega.Valor_pagado #variable Target 